# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [619]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [620]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [621]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [622]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):  # deprecated: see https://docs.python.org/2.7/library/xml.etree.elementtree.html
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [623]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

def make_df(root, tag, attribselector, oncolumn, comparisontype):
    elements = []
    clist = []

# grab all elements for tag (ie - city) and build a column list while we're here
    for e in root.iter(tag): 
        elements.append(e)
        for child in e.iter():
            clist.append(child.tag)
    
    # now lets have a unique list and make it dataframe friendly
    clist = list(set(clist))
    column_series = pd.Series(clist, index=range(len(clist)))
    
# make dataframe of appropriate size to accomodate all elements
    df = pd.DataFrame(index=range(len(elements)), columns=['name'])

# add columns
    # no matter what attribute we're checking against, 
    # we just need a place to store its value for comparison
    df['attribselector'] = None 
    df[tag] = None
    # now all the others
    for cs in column_series: 
        if (cs != 'name') & (cs != tag): df[cs] = None        
    
# fill dataframe
    i = 0
    for e in elements:
        # Sometimes we're looking for attributes in the primary element.
        if oncolumn == tag:
            df.loc[i]['attribselector'] = e.get(attribselector)
            
        # just making sure we always have the name, cuz it does the columns backwards and we need it below
        if "name" in column_series.values:
            curname = e.find('name')
            
        for thiscolumn in column_series:
            column_values = e.findall(thiscolumn)
            for thiscolumnvalue in column_values:
                if thiscolumnvalue != None:
                    ondeck = thiscolumnvalue.text
                    
                    # oncolumn is the column we want attributes from, but
                    # most of the time we're not doing that
                    if thiscolumn != oncolumn:
                        df.loc[i][thiscolumn] = ondeck
                    
                    # and as usual, the smallest number of cases require the most code
                    else:
                        # 1: if there's no attribute, maybe this is the only datapoint of its type
                        if (attribselector not in thiscolumnvalue.attrib):
                            df.loc[i][thiscolumn] = ondeck
                        
                        # 2: if the attribute is there, we should check against the previous attrib value
                        #    if one exists and place the current attrib value for later comparison
                        if (attribselector in thiscolumnvalue.attrib):
                            current_attrib_value = thiscolumnvalue.get(attribselector)
                            # our first encounter with the attribute
                            # so we'll use the value and place the attribute
                            if df.loc[i]['attribselector'] == None:  
                                df.loc[i][thiscolumn] = ondeck
                                df.loc[i]['attribselector'] = current_attrib_value
                            
                            # subsequent encounters with the attribute 
                            # so we have to check it against past values
                            elif df.loc[i]['attribselector'] != None: 
                                past_attrib_value = df.loc[i]['attribselector']
                                if comparisontype == "mostest":
                                    if current_attrib_value > past_attrib_value: 
                                        df.loc[i][thiscolumn] = ondeck
                                        df.loc[i]['attribselector'] = current_attrib_value
                                elif comparisontype == "enumerate":
                                    df = df.append(df.loc[i].to_dict(), ignore_index=True)
                                    df.loc[len(df) - 1]['name'] = curname.text
                                    df.loc[len(df) - 1][thiscolumn] = ondeck
                                    df.loc[len(df) - 1]['attribselector'] = current_attrib_value
        i = i + 1
    return df


In [626]:
### -- question 1: 10 countries with the lowest infant mortality rates

adf = make_df(root, tag="country", attribselector="na", oncolumn="na", comparisontype="na")
adf['infant_mortality'] = adf['infant_mortality'].astype('float')
adf[['name', 'infant_mortality']].sort_values('infant_mortality', ascending=True)[:10]

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [627]:
### -- question 2: 10 cities with the largest population

cdf = make_df(root, tag="city", attribselector="year", oncolumn="population", comparisontype="mostest")
cdf['population'] = cdf['population'].astype('float')
cdf[['name', 'population', 'attribselector']].sort_values('population', ascending=False)[0:10]

,name,population,attribselector
1341,Shanghai,22315474.0,2010
771,Istanbul,13710512.0,2012
1527,Mumbai,12442373.0,2011
479,Moscow,11979529.0,2013
1340,Beijing,11716620.0,2010
2810,São Paulo,11152344.0,2010
1342,Tianjin,11090314.0,2010
1064,Guangzhou,11071424.0,2010
1582,Delhi,11034555.0,2011
1067,Shenzhen,10358381.0,2010


In [632]:
### -- question 3: 10 ethnic groups with the largest overall populations 
### -- (sum of best/latest estimates over all countries)

# let's get country populations first
cpop = make_df(root, tag="country", attribselector="year", oncolumn="population", comparisontype="mostest")
cpop['population'] = cpop['population'].astype('float')

# now the ethnic percentages
edf = make_df(root, tag="country", attribselector="percentage", oncolumn="ethnicgroup", comparisontype="enumerate")
edf['attribselector'] = edf['attribselector'].astype('float')
edf['population'] = edf['population'].astype('float')

# use population values from cpop. 
listofcountries = edf.name.unique()
for con in listofcountries:
    cpopval = cpop.population.loc[cpop.name == con].iloc[0]
    edf.population.loc[edf.name == con] = cpopval

# adjust by percentages
edf.population = (edf.population * edf.attribselector)/100

listofethnicities = edf.ethnicgroup.unique()
summarytable = pd.DataFrame(index=range(len(listofethnicities)), columns=['ethnicgroup', 'population'])
summarytable.drop(summarytable.index, inplace=True)

i = 0
for eth in listofethnicities:
    thisgroup = edf[edf.ethnicgroup == eth]
    totpop = thisgroup.population.sum()
    summarytable = summarytable.append({'ethnicgroup':eth, 'population':totpop}, ignore_index=True)

summarytable.sort_values('population', ascending=False)[0:10]

,ethnicgroup,population
40,Han Chinese,1.245059e+09
159,Indo-Aryan,8.718156e+08
59,European,4.948722e+08
71,African,3.183251e+08
49,Dravidian,3.027137e+08
67,Mestizo,1.577344e+08
47,Bengali,1.467769e+08
19,Russian,1.318570e+08
62,Japanese,1.265342e+08
52,Malay,1.219936e+08


In [634]:
### -- question 4: name and country of:

# a) longest river
rivers = make_df(root, tag="river", attribselector="country", oncolumn="river", comparisontype="na")
rivers['length'] = rivers['length'].astype('float')
print rivers[['attribselector', 'name', 'length']].sort_values('length', ascending=False)[0:1]
print '\n'

# b) largest lake
lakes = make_df(root, tag="lake", attribselector="country", oncolumn="lake", comparisontype="na")
lakes['area'] = lakes['area'].astype('float')
print lakes[['attribselector', 'name', 'area']].sort_values('area', ascending=False)[0:1]
print '\n'

# c) airport at highest elevation
airports = make_df(root, tag="airport", attribselector="country", oncolumn="airport", comparisontype="na")
airports['elevation'] = airports['elevation'].astype('float')
print airports[['attribselector', 'name', 'elevation']].sort_values('elevation', ascending=False)[0:1]

    attribselector      name  length
174       CO BR PE  Amazonas  6448.0


    attribselector         name      area
54  R AZ KAZ IR TM  Caspian Sea  386400.0


   attribselector          name  elevation
80            BOL  El Alto Intl     4063.0
